## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Conclusion](#results)

### Introduction: Business Problem
In a city of New York, if someone is looking to open an Indian restaurant in Queens borough of New York. Recommend that whether they should open it or not based what type of restaurants and other venue items in Queens area. The intention is to open India restaurant in Queens borough beaause there is general knowledge and understandig that there is high  population of Indian community in Queens, NY.

We will try to review Queen's neighboorhood data and different venues around it and divide them in different clusters. 

### Data

Based on definition of our problem, factors that will influence our decission are:

the existing restaurants in the neighborhood (any type of restaurant)
the other venue items around which may help with traffic in Indian restaurnt
Eyeball the final results and provide conclustion

New York primary has 5 boroughs and 300+ neighborhoods. In order to determine if restaurt can be opened in Queens, NY, we will segment and explore the neighborhoods.  We will then use Foursquare API to explore the neighborhoods and determine the Queens, NY is a good location to open the Indian restaurant. We will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 


In [229]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [232]:
## Download and Explore Dataset ##
# Get NY data from the dataset
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('NY Data downloaded!')

NY Data downloaded!


In [233]:
# json format
with open('newyork_data.json') as json_data:
    ny_data = json.load(json_data)

In [234]:
#eye ball to see what needs to be done with it
ny_data 

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [235]:
# It is obvious that the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.
neighborhoods_feaures = ny_data['features']

In [236]:
# just see how data looks like 
neighborhoods_feaures[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [237]:
## Tranform the data into a pandas dataframe ##
# The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. 
# So let's start by creating an empty dataframe.

# we would use borough, neighboorhood, latitude and longtitue data
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_neighborhoods = pd.DataFrame(columns=column_names)
df_neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [238]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_neighborhoods = df_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [116]:
df_neighborhoods.shape

(307, 4)

In [239]:
#find out how many boroughs  and neigboorhoods, how many unique boroughs and make sure Queens exist in the list
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_neighborhoods['Borough'].unique()),
        df_neighborhoods.shape[0]
    )
)

# each brough will be used to get the stats  but Queens will be used to get the stats, as plan is to open in Queens and whether it is a good idea
print('Unique boroughs of NY', df_neighborhoods['Borough'].unique())

The dataframe has 5 boroughs and 306 neighborhoods.
Unique boroughs of NY ['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


In [240]:
## Use geopy library to get the latitude and longitude values of New York City.##
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [241]:
## Create a map of New York with neighborhoods superimposed on top. ##
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [177]:
# for our purpose let's simplify the above map and segment and cluster only the neighborhoods in Queens. 
# So let's slice the original dataframe and create a new dataframe for the Queens data.

queens_data = df_neighborhoods[df_neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [126]:
#Let's get the geographical coordinates of Manhattan
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


In [173]:
#print neighboorhoods of Queens
print('Unique neighboorhoods of Queens', queens_data['Neighborhood'].unique())

Unique neighboorhoods of Queens ['Astoria' 'Woodside' 'Jackson Heights' 'Elmhurst' 'Howard Beach' 'Corona'
 'Forest Hills' 'Kew Gardens' 'Richmond Hill' 'Flushing' 'Long Island City'
 'Sunnyside' 'East Elmhurst' 'Maspeth' 'Ridgewood' 'Glendale' 'Rego Park'
 'Woodhaven' 'Ozone Park' 'South Ozone Park' 'College Point' 'Whitestone'
 'Bayside' 'Auburndale' 'Little Neck' 'Douglaston' 'Glen Oaks' 'Bellerose'
 'Kew Gardens Hills' 'Fresh Meadows' 'Briarwood' 'Jamaica Center'
 'Oakland Gardens' 'Queens Village' 'Hollis' 'South Jamaica' 'St. Albans'
 'Rochdale' 'Springfield Gardens' 'Cambria Heights' 'Rosedale'
 'Far Rockaway' 'Broad Channel' 'Breezy Point' 'Steinway' 'Beechhurst'
 'Bay Terrace' 'Edgemere' 'Arverne' 'Rockaway Beach' 'Neponsit'
 'Murray Hill' 'Floral Park' 'Holliswood' 'Jamaica Estates'
 'Queensboro Hill' 'Hillcrest' 'Ravenswood' 'Lindenwood' 'Laurelton'
 'Lefrak City' 'Belle Harbor' 'Rockaway Park' 'Somerville' 'Brookville'
 'Bellaire' 'North Corona' 'Forest Hills Gardens' 'Jama

In [128]:
# let's visualizat Queens the neighborhoods in it.
# create map of Queens using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### Methodology 
Evaluate data within 5000 meter radius and top 100 venues

##### Use Fourqure API to explore the Queens neighboorhood and segment them.

In [160]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: K4QQTHDMJJZWLODASEWNLKEMRG334ZKQY0A0B4K1J3RAQV5S
CLIENT_SECRET:N05ZTAFZP3YQLYYUUI0C2U3BDBXOW2A20KNQTBV3ANV32R1Y


In [161]:
queens_data.loc[0, 'Neighborhood']
queens_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [162]:
## Explore Neighborhoods in Manhattan ##
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [163]:
def getNearbyVenues(neighboorhoodnames, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for neighboorhoodname, lat, lng in zip(neighboorhoodnames, latitudes, longitudes):
        #print(neighboorhoodname) #neighboorhood name
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighboorhoodname, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [164]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called queens_venues.
queens_venues = getNearbyVenues(neighboorhoodnames=queens_data['Neighborhood'],
                                latitudes=queens_data['Latitude'],
                                longitudes=queens_data['Longitude']
                                )


In [178]:
## check the size of the resulting dataframe ##
print(queens_venues.shape)
queens_venues.head()

(2145, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


In [166]:
#Let's check how many venues were returned for each neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,17,17,17,17,17,17
Astoria,100,100,100,100,100,100
Astoria Heights,14,14,14,14,14,14
Auburndale,21,21,21,21,21,21
Bay Terrace,41,41,41,41,41,41
Bayside,73,73,73,73,73,73
Bayswater,3,3,3,3,3,3
Beechhurst,18,18,18,18,18,18
Bellaire,11,11,11,11,11,11


In [179]:
#Let's find out how many unique categories can be curated from all the returned venues
# We will primarily be interested in restaurant category and probably other categories which help increasing the foot traffic to Indian restaurnt
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 263 uniques categories.


In [180]:
## Analyze Each Neighborhood

# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [181]:
#And let's examine the new dataframe size.
queens_onehot.shape

(2145, 263)

In [182]:
## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [183]:
queens_grouped.shape

(81, 263)

In [186]:
##Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 25

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
                      venue  freq
0                 Surf Spot  0.24
1             Metro Station  0.12
2           Thai Restaurant  0.06
3               Coffee Shop  0.06
4           Bed & Breakfast  0.06
5                 Wine Shop  0.06
6                  Bus Stop  0.06
7                Playground  0.06
8               Pizza Place  0.06
9                Donut Shop  0.06
10                    Beach  0.06
11           Sandwich Place  0.06
12               Board Shop  0.06
13             Optical Shop  0.00
14             Noodle House  0.00
15                Nightclub  0.00
16               Nail Salon  0.00
17  New American Restaurant  0.00
18              Music Venue  0.00
19                   Museum  0.00
20                Multiplex  0.00
21     Other Great Outdoors  0.00
22              Yoga Studio  0.00
23          Other Nightlife  0.00
24    Outdoors & Recreation  0.00


----Astoria----
                        venue  freq
0   Middle Eastern Restaurant  0.07
1         

In [187]:
##Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending ord
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


### Analysis
Analyze that 25 venues and create 10 clusters using kmeans.Review the data

In [188]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Beach,Wine Shop,Playground,Sandwich Place,Pizza Place,Board Shop,Thai Restaurant,Bus Stop,Donut Shop,Coffee Shop,Bed & Breakfast,Women's Store,Empanada Restaurant,Falafel Restaurant,Event Space,Egyptian Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Electronics Store,Dry Cleaner,Eastern European Restaurant
1,Astoria,Bar,Middle Eastern Restaurant,Hookah Bar,Greek Restaurant,Seafood Restaurant,Bakery,Italian Restaurant,Mediterranean Restaurant,Salon / Barbershop,Café,Pub,Coffee Shop,Deli / Bodega,Pizza Place,Latin American Restaurant,Food Truck,Indian Restaurant,Ice Cream Shop,Gourmet Shop,Grocery Store,Gym,Food & Drink Shop,Gym / Fitness Center,Japanese Restaurant,Salad Place
2,Astoria Heights,Italian Restaurant,Playground,Deli / Bodega,Bakery,Supermarket,Shopping Mall,Bus Station,Bowling Alley,Pizza Place,Hostel,Burger Joint,Plaza,Music Venue,Fast Food Restaurant,Farmers Market,Egyptian Restaurant,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Dry Cleaner,Eastern European Restaurant,Dumpling Restaurant,Fish & Chips Shop
3,Auburndale,Italian Restaurant,Hookah Bar,Gymnastics Gym,Furniture / Home Store,Noodle House,Bar,Discount Store,Mobile Phone Shop,Toy / Game Store,Supermarket,Train,Athletics & Sports,Miscellaneous Shop,Ice Cream Shop,Fast Food Restaurant,Deli / Bodega,American Restaurant,Pet Store,Pharmacy,Korean Restaurant,Farmers Market,Farm,Falafel Restaurant,Filipino Restaurant,Event Space
4,Bay Terrace,Clothing Store,Women's Store,Kids Store,Cosmetics Shop,Donut Shop,Mobile Phone Shop,Lingerie Store,American Restaurant,Furniture / Home Store,Gift Shop,Shoe Store,Shopping Mall,Video Game Store,Movie Theater,Sporting Goods Shop,Weight Loss Center,Steakhouse,Supermarket,Men's Store,Coffee Shop,Bus Stop,Bank,Accessories Store,Bakery,Gluten-free Restaurant


In [225]:
### Cluster Neighborhoods
# set number of clusters
kclusters = 10

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 4, 2, 0, 2], dtype=int32)

In [226]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [227]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [192]:
###Examine Clusters

#Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
#Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you

queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
15,Glendale,Food & Drink Shop,Brewery,Deli / Bodega,Arts & Crafts Store,Fast Food Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,Egyptian Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Electronics Store,Women's Store,Eastern European Restaurant,Food Court,Dry Cleaner,Donut Shop,Dog Run,Dive Bar,Discount Store
19,South Ozone Park,Deli / Bodega,Park,Hotel,Food Truck,Donut Shop,Fast Food Restaurant,Sandwich Place,Bar,Empanada Restaurant,Egyptian Restaurant,Electronics Store,Farm,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish & Chips Shop,Fish Market,Eastern European Restaurant,Dry Cleaner,Dumpling Restaurant,Food,Dog Run,Dive Bar,Discount Store
21,Whitestone,Bubble Tea Shop,Dance Studio,Deli / Bodega,Train,Women's Store,Farm,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Farmers Market,Eastern European Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Egyptian Restaurant,Dry Cleaner,Dumpling Restaurant,Food & Drink Shop,Donut Shop,Dog Run,Dive Bar,Discount Store
30,Briarwood,Deli / Bodega,Convenience Store,Arts & Crafts Store,Bus Station,Fast Food Restaurant,Coffee Shop,Gym,Mexican Restaurant,Indian Restaurant,Empanada Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Cycle Studio,Dance Studio,Farmers Market,Farm,Falafel Restaurant,Event Space,Dessert Shop,Dim Sum Restaurant,Egyptian Restaurant,Eastern European Restaurant
42,Broad Channel,Other Nightlife,Pizza Place,Deli / Bodega,Dive Bar,Bus Station,Farm,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Eastern European Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Egyptian Restaurant,Women's Store,Dumpling Restaurant,Cosmetics Shop,Dry Cleaner,Donut Shop,Dog Run,Discount Store
65,Bellaire,Deli / Bodega,Convenience Store,Italian Restaurant,Moving Target,Bus Station,Bus Stop,Chinese Restaurant,Greek Restaurant,Coffee Shop,Grocery Store,Discount Store,Empanada Restaurant,Fish & Chips Shop,Cycle Studio,Filipino Restaurant,Fast Food Restaurant,Dance Studio,Department Store,Farmers Market,Farm,Falafel Restaurant,Event Space,Electronics Store,Dive Bar,Egyptian Restaurant


In [193]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
63,Somerville,Park,Women's Store,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Egyptian Restaurant,Eastern European Restaurant,Dry Cleaner,Food & Drink Shop,Donut Shop,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [194]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Astoria,Bar,Middle Eastern Restaurant,Hookah Bar,Greek Restaurant,Seafood Restaurant,Bakery,Italian Restaurant,Mediterranean Restaurant,Salon / Barbershop,Café,Pub,Coffee Shop,Deli / Bodega,Pizza Place,Latin American Restaurant,Food Truck,Indian Restaurant,Ice Cream Shop,Gourmet Shop,Grocery Store,Gym,Food & Drink Shop,Gym / Fitness Center,Japanese Restaurant,Salad Place
1,Woodside,Grocery Store,Deli / Bodega,Bakery,Thai Restaurant,Latin American Restaurant,Filipino Restaurant,American Restaurant,Bar,Pizza Place,Donut Shop,Pub,Ice Cream Shop,Sandwich Place,Café,Restaurant,Mexican Restaurant,Chinese Restaurant,Gourmet Shop,Gym / Fitness Center,Pharmacy,Spa,Metro Station,Mobile Phone Shop,Empanada Restaurant,Coffee Shop
2,Jackson Heights,Latin American Restaurant,South American Restaurant,Peruvian Restaurant,Mobile Phone Shop,Bakery,Thai Restaurant,Spanish Restaurant,Mexican Restaurant,Pizza Place,Clothing Store,Diner,Empanada Restaurant,Supplement Shop,Grocery Store,Pharmacy,School,Cuban Restaurant,Kids Store,Laundromat,Shoe Store,Shipping Store,Bubble Tea Shop,Burger Joint,Cajun / Creole Restaurant,Lounge
3,Elmhurst,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Indonesian Restaurant,Bubble Tea Shop,Vietnamese Restaurant,South American Restaurant,Malay Restaurant,Sushi Restaurant,Donut Shop,Bar,Bank,Colombian Restaurant,Gym / Fitness Center,Snack Place,Ice Cream Shop,Hotpot Restaurant,Park,Argentinian Restaurant,Food,Pizza Place,Salon / Barbershop,Egyptian Restaurant,Farmers Market,Farm
4,Howard Beach,Italian Restaurant,Bagel Shop,Fast Food Restaurant,Pharmacy,Sandwich Place,Chinese Restaurant,Donut Shop,Tapas Restaurant,Diner,Sushi Restaurant,Supermarket,Cosmetics Shop,Fried Chicken Joint,Japanese Restaurant,Spa,Breakfast Spot,Mexican Restaurant,Seafood Restaurant,Sporting Goods Shop,Clothing Store,Bank,Deli / Bodega,Convenience Store,Construction & Landscaping,Gym / Fitness Center
5,Corona,Mexican Restaurant,Pizza Place,Italian Restaurant,Convenience Store,Food & Drink Shop,Empanada Restaurant,Supermarket,School,Sandwich Place,Donut Shop,Bakery,Restaurant,South American Restaurant,Park,Deli / Bodega,Ice Cream Shop,Diner,Dance Studio,Cosmetics Shop,Fast Food Restaurant,Creperie,Farmers Market,Cuban Restaurant,Farm,Falafel Restaurant
6,Forest Hills,Gym,Gym / Fitness Center,Pizza Place,Thai Restaurant,Park,Yoga Studio,Pharmacy,Convenience Store,Business Service,Sandwich Place,Liquor Store,Italian Restaurant,Snack Place,Peruvian Restaurant,Spa,Food Truck,Donut Shop,Chinese Restaurant,Supermarket,Supplement Shop,Farmers Market,Bakery,Bagel Shop,Gift Shop,Optical Shop
7,Kew Gardens,Chinese Restaurant,Park,Pharmacy,Pizza Place,Indian Restaurant,Donut Shop,Bank,Bar,Deli / Bodega,South American Restaurant,Food,Spa,Bubble Tea Shop,Pub,Juice Bar,Diner,Fish Market,Sandwich Place,Italian Restaurant,New American Restaurant,Movie Theater,Sushi Restaurant,Bakery,Trail,Wine Shop
8,Richmond Hill,Pizza Place,Lounge,Latin American Restaurant,Discount Store,Spanish Restaurant,Bus Station,Supermarket,Caribbean Restaurant,Sandwich Place,Bank,Donut Shop,Clothing Store,Metro Station,Diner,Gym / Fitness Center,Indian Restaurant,Pet Service,Empanada Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Cuban Restaurant,Cycle Studio,Farm,Falafel Restaurant
9,Flushing,Chinese Restaurant,Hotpot Restaurant,Spa,Bubble Tea Shop,Karaoke Bar,Korean Restaurant,Asian Res

In [195]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
43,Breezy Point,Beach,American Restaurant,Board Shop,Monument / Landmark,Trail,Women's Store,Farmers Market,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Filipino Restaurant,Fast Food Restaurant,Egyptian Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Electronics Store,Eastern European Restaurant,Food Court,Dumpling Restaurant,Dry Cleaner,Donut Shop,Dog Run
78,Hammels,Beach,Southern / Soul Food Restaurant,Diner,Café,Shoe Store,Gym / Fitness Center,Dog Run,Bus Stop,Bus Station,Fast Food Restaurant,Deli / Bodega,Food Truck,Farmers Market,Empanada Restaurant,Filipino Restaurant,Farm,Fish & Chips Shop,Falafel Restaurant,Fish Market,Event Space,Women's Store,Electronics Store,Flower Shop,Eastern European Restaurant,Dumpling Restaurant


In [197]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
77,Malba,Rest Area,Tennis Court,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Dumpling Restaurant,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Egyptian Restaurant,Eastern European Restaurant,Women's Store,Dry Cleaner,Food & Drink Shop,Donut Shop,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant
79,Bayswater,Tennis Court,Playground,Park,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Women's Store,Dumpling Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Farm,Donut Shop,Dry Cleaner,Food,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant


In [198]:
queens_merged.loc[queens_merged['Cluster Labels'] == 5, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
64,Brookville,Deli / Bodega,Women's Store,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Egyptian Restaurant,Dumpling Restaurant,Food Court,Dry Cleaner,Donut Shop,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant


In [199]:
queens_merged.loc[queens_merged['Cluster Labels'] == 6, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
59,Laurelton,Caribbean Restaurant,Cosmetics Shop,Train Station,Park,Sculpture Garden,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Dry Cleaner,Empanada Restaurant,Filipino Restaurant,Electronics Store,Fish & Chips Shop,Egyptian Restaurant,Eastern European Restaurant,Fish Market,Dumpling Restaurant,Event Space,Donut Shop,Food,Dog Run,Dive Bar,Discount Store,Diner


In [200]:
queens_merged.loc[queens_merged['Cluster Labels'] == 7, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
35,South Jamaica,Bus Station,Ice Cream Shop,Pizza Place,Home Service,Bakery,Grocery Store,Bus Line,Caribbean Restaurant,Supermarket,Women's Store,Event Space,Electronics Store,Empanada Restaurant,Farmers Market,Falafel Restaurant,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Egyptian Restaurant,Dry Cleaner,Eastern European Restaurant,Dumpling Restaurant,Flower Shop,Donut Shop
53,Holliswood,Bus Station,Playground,Mobile Phone Shop,Donut Shop,Supermarket,Women's Store,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Farm,Eastern European Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Egyptian Restaurant,Dumpling Restaurant,Food,Dry Cleaner,Dog Run,Dive Bar,Discount Store,Diner
70,Pomonok,Bus Station,Supermarket,Japanese Restaurant,Shop & Service,Scenic Lookout,Bar,Bowling Alley,Playground,Pizza Place,Fast Food Restaurant,Farmers Market,Filipino Restaurant,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Egyptian Restaurant,Eastern European Restaurant,Women's Store,Dumpling Restaurant,Fish Market,Dry Cleaner,Donut Shop,Dog Run


In [201]:
queens_merged.loc[queens_merged['Cluster Labels'] == 8, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
51,Murray Hill,Korean Restaurant,Coffee Shop,Supermarket,Bar,Bank,Breakfast Spot,Bakery,Pet Service,Cosmetics Shop,Dessert Shop,Pub,Mobile Phone Shop,Fish & Chips Shop,Fried Chicken Joint,Karaoke Bar,Bath House,Pizza Place,Japanese Restaurant,BBQ Joint,Automotive Shop,Intersection,Donut Shop,Farmers Market,Farm,Falafel Restaurant


In [202]:
queens_merged.loc[queens_merged['Cluster Labels'] == 9, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
50,Neponsit,Beach,Women's Store,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Egyptian Restaurant,Dumpling Restaurant,Food Court,Dry Cleaner,Donut Shop,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant


In [203]:
queens_merged.loc[queens_merged['Cluster Labels'] == 10, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue


### Results and Conclusion
Above analsysis should have shown a good amount of Indian restuarnts in top 25 venues and 10 cluster. But that does not seem to be case. The cluster data and top venues indicate that there is definitely an opporutity to open up an Indian restuarnt in Queens borough. There are enough venues which can also increase the foot traffic to the Indian restaurnt
